# Visualizing MISP Data within Neo4j for Link Analysis

This notebook visualizes MISP Events within a Neo4j database. 

## Initial Setup

Configuring MISP and Neo4j credentials

In [1]:
from pymisp import PyMISP
from neo4j import GraphDatabase
import jmespath

# MISP configuration
MISP_URL = 'https://localhost'
MISP_KEY = '53XxW3buXZsFqLm0Mqry8sGQCsNlSDfKfKmX7jei'

# Neo4j configuration
NEO4J_URL = 'bolt://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'disco-ventura-torso-robin-carrot-8913'

## Function for reading a MISP event and pushing it to Neo4j

In [26]:
def create_event_nodes(tx, event):
       # Create the event node
       print(f"Creating Event: {event['info']}")
       tx.run("MERGE (e:Event {id: $id}) "
              "SET e.info = $info, e.date = $date", id=event['uuid'], info=event['info'], date=event['date'])

       # Create the attribute nodes and relationships
       for attribute in event['Attribute']:
              tx.run("MERGE (a:"+ attribute['type'].replace('-', '_') + " {id: $id}) "
                     "SET a.type = $type, a.value = $value, a.category = $category", id=attribute['value'],
                     type=attribute['type'], value=attribute['value'], category=attribute['category'])
              
              # Create the relationship between the event and attribute
              tx.run("MATCH (e:Event {id: $eventId}), (a:" + attribute['type'].replace('-', '_') + " {id: $attributeId}) "
                     "MERGE (e)-[:HAS_ATTRIBUTE]->(a)", eventId=event['uuid'], attributeId=attribute['value'])
       
       for obj in event['Object']:
              objectId = obj['uuid']
              # Create the relationship between the event and object
              tx.run("MATCH (e:Event {id: $eventId}), (o:Object {id: $objectId, comment: $objectComment}) "
                     "MERGE (e)-[:HAS_OBJECT]->(o)", 
                     eventId=event['uuid'], 
                     objectId=objectId, 
                     objectComment=obj['comment']
              )
              
              # Create the relationships b/w the object and object attributes
              objectRefAttrs = jmespath.search('[].Attribute.{value: value, type: type}', obj['ObjectReference'])
              for attribute in obj['Attribute']:
                     tx.run("MERGE (a:" + attribute['type'].replace('-', '_') + " {id: $id}) "
                     "SET a.type = $type, a.value = $value, a.category = $category", id=attribute['value'],
                     type=attribute['type'], value=attribute['value'], category=attribute['category'])
                     # Create the relationship between the event and attribute
                     # tx.run("MATCH (e:Event {id: $eventId}), (a:Attribute {id: $attributeId}) "
                     #        "MERGE (e)-[:HAS_ATTRIBUTE]->(a)", eventId=event['uuid'], attributeId=attribute['value'])

                     # tx.run("MATCH (o:Object {id: $objectId}), (a:Attribute {id: $attributeId}) "
                     #        "MERGE (o)-[:HAS_ATTRIBUTE]->(a)", objectId=objectId, attributeId=attribute['value'])
                     for refAttr in objectRefAttrs:
                            if refAttr['value'] != attribute['value']:
                                   print(f"{attribute['value']} --> {refAttr}")
                                   tx.run("MATCH (a:" + attribute['type'].replace('-', '_') + " {id: $attributeId}), (ra:" + refAttr['type'].replace('-', '_') + "{id: $refAttrId, type: $refAttrType})"
                                          "MERGE (a)-[:RELATED_TO]->(ra)", 
                                          refAttrId=refAttr['value'], 
                                          attributeId=attribute['value'], 
                                          type=attribute['type'], 
                                          refAttrType=refAttr['type'])
                            



In [3]:
# Execute the transaction to delete all nodes
def delete_all_nodes(tx):
    tx.run("MATCH (n) DETACH DELETE n")

In [4]:
# Connect to MISP
misp = PyMISP(MISP_URL, MISP_KEY, ssl=False)

/home/bikram/Git/cti-platform/orchestration/venv/lib64/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
The version of PyMISP recommended by the MISP instance (2.4.171) is newer than the one you're using now (2.4.170.2). Please upgrade PyMISP.
/home/bikram/Git/cti-platform/orchestration/venv/lib64/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/bikram/Git/cti-platform/orchestration/venv/lib64/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being 

In [5]:
# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [8]:
# Get a particular event
event = misp.get_event(7)


/home/bikram/Git/cti-platform/orchestration/venv/lib64/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
# Fetch events from MISP
events = misp.search()

In [ ]:
events

In [28]:
# Create nodes and relationships in Neo4j
events = [event]
with driver.session() as session:
    session.write_transaction(lambda tx: [create_event_nodes(tx, event['Event']) for event in events])

# Close the Neo4j driver
driver.close()

/tmp/ipykernel_93180/2162331495.py:4: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(lambda tx: [create_event_nodes(tx, event['Event']) for event in events])


Creating Event: Bluenoroff’s RustBucket campaign
13768 --> {'value': '104.255.172.52', 'type': 'ip-dst'}
Los Angeles --> {'value': '104.255.172.52', 'type': 'ip-dst'}
US --> {'value': '104.255.172.52', 'type': 'ip-dst'}
United States --> {'value': '104.255.172.52', 'type': 'ip-dst'}
Aptum Technologies --> {'value': '104.255.172.52', 'type': 'ip-dst'}
34.05223 --> {'value': '104.255.172.52', 'type': 'ip-dst'}
-118.24368 --> {'value': '104.255.172.52', 'type': 'ip-dst'}
H4Y Technologies LLC --> {'value': '104.255.172.52', 'type': 'ip-dst'}
CA --> {'value': '104.255.172.52', 'type': 'ip-dst'}
80 --> {'value': '104.255.172.52', 'type': 'ip-dst'}
443 --> {'value': '104.255.172.52', 'type': 'ip-dst'}
13098529066745705731 --> {'value': '104.255.172.52', 'type': 'ip-dst'}
sha1WithRSAEncryption --> {'value': '104.255.172.52', 'type': 'ip-dst'}
/CN=localhost --> {'value': '104.255.172.52', 'type': 'ip-dst'}
/CN=localhost --> {'value': '104.255.172.52', 'type': 'ip-dst'}
2019-11-08T23:48:47.00000

In [27]:
# Run the transaction
with driver.session() as session:
    session.write_transaction(delete_all_nodes)

/tmp/ipykernel_93180/922533380.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(delete_all_nodes)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# Create an empty graph
G = nx.Graph()

# Create nodes for Event, Attributes, Galaxies, and Objects
G.add_node(event['Event']['uuid'], label='Event', info=event['Event']['info'])
for attribute in event['Event']['Attribute']:
    G.add_node(attribute['uuid'], label='Attribute', type=attribute['type'], value=attribute['value'])
for galaxy in event['Event']['Galaxy']:
    G.add_node(galaxy['uuid'], label='Galaxy', name=galaxy['name'])
for obj in event['Event']['Object']:
    G.add_node(obj['uuid'], label='Object', name=obj['name'])

# Create relationships
for attribute in event['Event']['Attribute']:
    G.add_edge(event['Event']['uuid'], attribute['value'], relationship='Event to Attribute')
    for galaxy in attribute['Galaxy']:
        G.add_edge(attribute['uuid'], galaxy['value'], relationship='Attribute to Galaxy')

# Visualize the graph
pos = nx.spring_layout(G)
node_labels = nx.get_node_attributes(G, 'label')
edge_labels = nx.get_edge_attributes(G, 'relationship')

plt.figure(figsize=(10, 10))
nx.draw_networkx(G, pos, with_labels=True, labels=node_labels)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.title('MISP Event Visualization')
plt.axis('off')
plt.show()


In [ ]:
# Create a tree layout
pos = nx.nx_pydot.graphviz_layout(G, prog='dot')

# Visualize the tree
plt.figure(figsize=(10, 100))
nx.draw(G, pos, with_labels=True, node_size=2000, node_shape='s', alpha=0.8, font_size=10)
edge_labels = nx.get_edge_attributes(G, 'relationship')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.title('MISP Event Tree Visualization')
plt.axis('off')
plt.show()